In [219]:
import pandas as pd
from unidecode import unidecode

In [220]:
df = pd.read_csv("../Data/Processed/crimes_clear.csv")

In [221]:
# Changing the field "fecha" into a datetime type
df['fecha'] = pd.to_datetime(df['fecha'])

# Adding a new column to know the day of the week that was reporte the crime
df['dia'] = df['fecha'].dt.day_name()

# Changing names to spanish since the whole database is in spanish
day_name_mapping = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes',
    'Wednesday': 'Miércoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

df['dia'] = df['dia'].map(day_name_mapping)

In [222]:
df.to_csv("../Data/Processed/crimes_enriched.csv", index=False)

# Importing and cleaning the new DF to know the location

In [223]:
position_df = pd.read_excel("../Data/Raw/DIVIPOLA_Municipios.xlsx", skiprows=10, skipfooter=10)

In [224]:
position_df = position_df.drop(['Código ', 'Código .1', 'Unnamed: 4'], axis=1)

In [225]:
position_df.columns

Index(['Nombre', 'Nombre.1', 'LATITUD', 'LONGITUD'], dtype='object')

In [226]:
# Remove the brackets and make text uppercase to match with the new DF
df.municipio = df.municipio.str.replace(r'\(.*\)', '').str.upper()

# Change the column's name to match in both dataframes
position_df.rename(columns={"Nombre.1": "municipio"}, inplace=True)

/tmp/ipykernel_250/150562150.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.municipio = df.municipio.str.replace(r'\(.*\)', '').str.upper()


In [227]:
df

,arma,departamento,municipio,fecha,genero,grupo_etario,codigo_dane,cantidad,delito,dia
0,ARMA BLANCA / CORTOPUNZANTE,AMAZONAS,LETICIA,2023-06-13,MASCULINO,ADULTOS,91001000,1,amenza,Martes
1,ARMA BLANCA / CORTOPUNZANTE,ANTIOQUIA,AMAGÁ,2023-06-26,FEMENINO,ADULTOS,5030000,1,amenza,Lunes
2,ARMA BLANCA / CORTOPUNZANTE,ANTIOQUIA,ANGOSTURA,2023-05-05,MASCULINO,ADULTOS,5038000,1,amenza,Viernes
3,ARMA BLANCA / CORTOPUNZANTE,ANTIOQUIA,BELLO,2023-02-23,FEMENINO,ADULTOS,5088000,1,amenza,Jueves
4,ARMA BLANCA / CORTOPUNZANTE,ANTIOQUIA,CISNEROS,2023-07-25,FEMENINO,ADULTOS,5190000,1,amenza,Martes
...,...,...,...,...,...,...,...,...,...,...
295881,SIN EMPLEO DE ARMAS,VICHADA,PUERTO CARREÑO,2023-08-27,FEMENINO,ADULTOS,99001000,1,violencia intrafamiliar,Domingo
295882,SIN EMPLEO DE ARMAS,VICHADA,SANTA ROSALÍA,2023-02-07,FEMENINO,ADULTOS,99624000,1,violencia intrafamiliar,Martes
295883,SIN EMPLEO DE ARMAS,VICHADA,SANTA ROSALÍA,2023-03-17,FEMENINO,ADULTOS,99624000,1,violencia intrafamiliar,Viernes
295884,SIN EMPLEO DE ARMAS,VICHADA,SANTA ROSALÍA,2023-03-31,FEMENINO,ADOLESCENTES,99624000,1,violencia intrafamiliar,Viernes


In [228]:
# Some places has an space at the end, this code remove the las space
df["municipio"] = df["municipio"].apply(lambda x: x[:-1] if x.endswith(" ") else x)

# Remove spanish accents and ticks
df["municipio"] =  df["municipio"].apply(unidecode)
position_df["municipio"] = position_df["municipio"].apply(unidecode)

In [229]:
# some places doesnt have athe same name, solvind this
places_to_change = {
"DON MATIAS" : "DONMATIAS",
"SAN VICENTE" : "SAN VICENTE FERRER",
"CARTAGENA" : "CARTAGENA DE INDIAS",
"MOMPOS" : "SANTA CRUZ DE MOMPOX",
"SAN ANDRES SOTAVENTO" : "SAN ANDRES DE SOTAVENTO",
"BOGOTA D.C." : "BOGOTA, D.C.",
"CUCUTA" : "SAN JOSE DE CUCUTA",
"CALI" : "SANTIAGO DE CALI",
"SANTAFE DE ANTIOQUIA" : "SANTA FE DE ANTIOQUIA",
"GUICAN" : "GUICAN DE LA SIERRA",
"CERRO SAN ANTONIO" : "CERRO DE SAN ANTONIO",
"MARIQUITA" : "SAN SEBASTIAN DE MARIQUITA",
"PIENDAMO" : "PIENDAMO - TUNIA",
"SOTARA" : "SOTARA PAISPAMBA",
"SAN JUAN DE RIO SECO" : "SAN JUAN DE RIOSECO",
"CUASPUD" : "CUASPUD CARLOSAMA",
"TOLU VIEJO" : "SAN JOSE DE TOLUVIEJO",
"LOPEZ" : "LOPEZ DE MICAY"}

df["municipio"] = df["municipio"].replace(places_to_change)

In [230]:
df[df["municipio"] == "PURISIMA"]["LATITUD"] = 9.238112
df[df["municipio"] == "PURISIMA"]["LONGITUD"] = -75.722401

/tmp/ipykernel_250/2056926640.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df["municipio"] == "PURISIMA"]["LATITUD"] = 9.238112
/tmp/ipykernel_250/2056926640.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df["municipio"] == "PURISIMA"]["LONGITUD"] = -75.722401


In [231]:
df = df.merge(position_df[["municipio" , "LATITUD", "LONGITUD"]], on="municipio", how="left")
df

,arma,departamento,municipio,fecha,genero,grupo_etario,codigo_dane,cantidad,delito,dia,LATITUD,LONGITUD
0,ARMA BLANCA / CORTOPUNZANTE,AMAZONAS,LETICIA,2023-06-13,MASCULINO,ADULTOS,91001000,1,amenza,Martes,-3.530059,-70.045137
1,ARMA BLANCA / CORTOPUNZANTE,ANTIOQUIA,AMAGA,2023-06-26,FEMENINO,ADULTOS,5030000,1,amenza,Lunes,6.032922,-75.708003
2,ARMA BLANCA / CORTOPUNZANTE,ANTIOQUIA,ANGOSTURA,2023-05-05,MASCULINO,ADULTOS,5038000,1,amenza,Viernes,6.861499,-75.358316
3,ARMA BLANCA / CORTOPUNZANTE,ANTIOQUIA,BELLO,2023-02-23,FEMENINO,ADULTOS,5088000,1,amenza,Jueves,6.358841,-75.589736
4,ARMA BLANCA / CORTOPUNZANTE,ANTIOQUIA,CISNEROS,2023-07-25,FEMENINO,ADULTOS,5190000,1,amenza,Martes,6.550065,-75.082541
...,...,...,...,...,...,...,...,...,...,...,...,...
322781,SIN EMPLEO DE ARMAS,VICHADA,PUERTO CARRENO,2023-08-27,FEMENINO,ADULTOS,99001000,1,violencia intrafamiliar,Domingo,5.836530,-68.141222
322782,SIN EMPLEO DE ARMAS,VICHADA,SANTA ROSALIA,2023-02-07,FEMENINO,ADULTOS,99624000,1,violencia intrafamiliar,Martes,4.968581,-70.659971
322783,SIN EMPLEO DE ARMAS,VICHADA,SANTA ROSALIA,2023-03-17,FEMENINO,ADULTOS,99624000,1,violencia intrafamiliar,Viernes,4.968581,-70.659971
322784,SIN EMPLEO DE ARMAS,VICHADA,SANTA ROSALIA,2023-03-31,FEMENINO,ADOLESCENTES,99624000,1,violencia intrafamiliar,Viernes,4.968581,-70.659971


In [233]:
df[df["municipio"] == "PURISIMA"]

,arma,departamento,municipio,fecha,genero,grupo_etario,codigo_dane,cantidad,delito,dia,LATITUD,LONGITUD
10832,SIN EMPLEO DE ARMAS,CÓRDOBA,PURISIMA,2023-03-29,MASCULINO,ADULTOS,23586000,1,amenza,Miércoles,NaN,NaN
10833,SIN EMPLEO DE ARMAS,CÓRDOBA,PURISIMA,2023-04-03,MASCULINO,ADULTOS,23586000,1,amenza,Lunes,NaN,NaN
10834,SIN EMPLEO DE ARMAS,CÓRDOBA,PURISIMA,2023-05-08,MASCULINO,ADULTOS,23586000,1,amenza,Lunes,NaN,NaN
10835,SIN EMPLEO DE ARMAS,CÓRDOBA,PURISIMA,2023-05-09,FEMENINO,ADULTOS,23586000,1,amenza,Martes,NaN,NaN
10836,SIN EMPLEO DE ARMAS,CÓRDOBA,PURISIMA,2023-05-24,FEMENINO,ADULTOS,23586000,1,amenza,Miércoles,NaN,NaN
10837,SIN EMPLEO DE ARMAS,CÓRDOBA,PURISIMA,2023-06-04,MASCULINO,ADULTOS,23586000,1,amenza,Domingo,NaN,NaN
10838,SIN EMPLEO DE ARMAS,CÓRDOBA,PURISIMA,2023-06-15,MASCULINO,ADULTOS,23586000,1,amenza,Jueves,NaN,NaN
10839,SIN EMPLEO DE ARMAS,CÓRDOBA,PURISIMA,2023-07-03,MASCULINO,ADULTOS,23586000,1,amenza,Lunes,NaN,NaN
10840,SIN EMPLEO DE ARMAS,CÓRDOBA,PURISIMA,2023-07-17,MASCULINO,ADULTOS,23586000,1,amenza,Lunes,NaN,NaN
10841,SIN EMPLEO DE ARMAS,CÓRDOBA,PURISIMA,2023-08-08,FEMENINO,ADULTOS,23586000,1,amenza,Martes,NaN,NaN


In [ ]:
position_df[position_df["municipio"] == "PIENDAMO - TUNIA"]

,Nombre,municipio,LATITUD,LONGITUD
387,CAUCA,PIENDAMO - TUNIA,2.70242,-76.537705


In [ ]:
# Places that wasnt find in the merge
df[df["LATITUD"].isnull()]["municipio"].unique()

array(['PURISIMA', '-'], dtype=object)

In [ ]:
"PURISIMA" : "",


SyntaxError: illegal target for annotation (1658178387.py, line 1)